# Importing the large datasets to a postgresql server

It is not possible to load the larger data sets in the memory of a local machine therefeore an alternative is to import them to a psql table and query them from there. By adding the right indices this can make the queries fast enough. After this import one can extract some basic statistics using sql and also export smaller portions of the data which can be handled by spark or pandas on a local machine.

## Unzipping the data and converting it to csv format

Unfortunately psql does not support an import of record json files therefore we need to convert the data sets to csv. We use here the command line tool [json2csv](https://github.com/jehiah/json2csv).

In [2]:
!gunzip ./data/large-datasets/reviews_CDs_and_Vinyl_5.json.gz ./data/large-datasets/reviews_CDs_and_Vinyl_5.json

gzip: ./data/large-datasets/reviews_CDs_and_Vinyl_5.json: unknown suffix -- ignored
rm: cannot remove 'reviews_CDs_and_Vinyl_5': No such file or directory


In [6]:
!json2csv -p -d '|' -k asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime -i ./data/large-datasets/reviews_CDs_and_Vinyl_5.json -o ./data/large-datasets/reviews_CDs_and_Vinyl_5.csv

## Importing the data in psql

To import the data in psql we create a table with the appropriate shape and import form the csv files generated above.

### Some preparation to run psql transactions and queries in python

In [61]:
import psycopg2 as pg
import pandas as pd

db_conf = { 
    'user': 'mariosk',
    'database': 'amazon_reviews'
}

connection_factory = lambda: pg.connect(user=db_conf['user'], database=db_conf['database'])

def transaction(*statements):
    try:
        connection = connection_factory()
        cursor = connection.cursor()
        for statement in statements:
            cursor.execute(statement)
        connection.commit()
        cursor.close()
    except pg.DatabaseError as error:
        print(error)
    finally:
        if connection is not None:
            connection.close()
    
def query(statement):
    try:
        connection = connection_factory()
        cursor = connection.cursor()
        cursor.execute(statement)
        
        header = [ description[0] for description in cursor.description ]
        rows = cursor.fetchall()
        
        cursor.close()
        return pd.DataFrame.from_records(rows, columns=header)
    except (Exception, pg.DatabaseError) as error:
        print(error)
        return None
    finally:
        if connection is not None:
            connection.close()

### Creating tables for with indices for the large datasets

In [30]:
transaction(
    'create table cds (asin text, helpful text, overall double precision, reviewText text, reviewTime text, reviewerID text, reviewerName text, summary text, unixReviewTime int);',
    'create index asin ON cds (asin);',
    'create index overall ON cds (overall);',
    'create index reviewerID ON cds (reviewerID);',
    'create index unixReviewTime ON cds (unixReviewTime);')

### Importing the datasets to psql

In [46]:
!psql -U mariosk -d amazon_reviews -c "\copy cds from './data/large-datasets/reviews_CDs_and_Vinyl_5.csv' with (format csv, delimiter '|', header true);"

COPY 1097592


## Querying the metrics

In [122]:
average_reviews_per_product = query('''
    with distinct_products as (select count(distinct asin) as products from cds),
         reviews_count as (select cast(count(*) as double precision) as reviews from cds)
    select reviews / products as reviews_per_product
    from distinct_products cross join reviews_count
''').rename(index={ 0: 'row'})

In [121]:
average_reviews_per_reviewer = query('''
    with distinct_reviewers as (select count(distinct reviewerID) as reviewers from cds),
         reviews_count as (select cast(count(*) as double precision) as reviews from cds)
    select reviews / reviewers as reviews_per_reviewer
    from distinct_reviewers cross join reviews_count
''').rename(index={ 0: 'row'})

In [119]:
percentages_per_rating = (query('''
    with rating_counts as (select count(overall) as rating_count from cds group by overall),
         reviews_count as (select cast(count(*) as double precision) as reviews from cds)
    select rating_count / reviews as row
    from rating_counts cross join reviews_count
    ''')
 .transpose()
 .rename(columns=lambda x: str(x + 1)))

In [123]:
all_metrics = pd.concat([percentages_per_rating, average_reviews_per_product, average_reviews_per_reviewer], axis=1)

In [124]:
all_metrics

,1,2,3,4,5,reviews_per_product,reviews_per_reviewer
row,0.224424,0.04243,0.042088,0.09277,0.598288,17.031982,14.58439
